In [59]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function

from datetime import date
import datetime
#imports

# Ideas

Raise error when len does not match before and after merging datasets (means that a player wasn't found -- likely because he is a junior or has two last names, etc.)

# To Do

1. Address last name issue
2. Get pitcher xba onto
3. Scrape data instead of downloading CSVs

ON avoiding duplicate names --- maybe could change xba column in pitchers df before I merge, that way any batters with the name Luis Garcia would not have the column and be left out

In [60]:
batters_df = pd.read_csv('../../data/individual_batting_stats.csv')
pitchers_df = pd.read_csv('../../data/individual_pitching_stats.csv')
misc_batters = pd.read_csv('../../data/batters_teams.csv')
misc_pitchers = pd.read_csv('../../data/pitchers_teams.csv')

Pitcher Savant URL

https://baseballsavant.mlb.com/leaderboard/custom?year=2022&type=pitcher&filter=&sort=4&sortDir=asc&min=25&selections=p_total_pa,p_ab,p_total_hits,p_strikeout,p_walk,p_k_percent,p_bb_percent,batting_avg,slg_percent,on_base_percent,xba,xslg,woba,xwoba,xobp,xiso,wobacon,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,&chart=false&x=xba&y=xba&r=no&chartType=beeswarm

Batter Savant URL 

https://baseballsavant.mlb.com/leaderboard/custom?year=2022&type=batter&filter=&sort=1&sortDir=asc&min=25&selections=b_ab,b_total_pa,b_total_hits,b_strikeout,b_walk,b_k_percent,b_bb_percent,batting_avg,slg_percent,on_base_percent,xba,xslg,woba,xwoba,xobp,xiso,wobacon,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,&chart=false&x=xba&y=xba&r=no&chartType=beeswarm

In [61]:
y = batters_df[batters_df['last_name'].str.contains(" ")]
batters_df['last_name'] = [i.split(' ')[0] for i in batters_df['last_name']]
#The teams df does not have Jr., III for example listed on the names so I will cut them off before that so they match

In [62]:
len(batters_df)

445

# Probable Pitchers

In [63]:
d = datetime.datetime.now()
today = d.strftime("%Y-%m-%d")
#Get today's date

In [64]:
url = f'https://www.mlb.com/probable-pitchers/{today}'
#URL of where I will get the pitchers' names

In [65]:
res = requests.get(url)
soup = BeautifulSoup(res.content)
#Making request to the site and storing it in soup

In [66]:
# pitchers = soup.find_all('div', {'class': 'probable-pitchers__pitcher-name'})

In [67]:
pitchers = soup.find_all('div', {'class': 'probable-pitchers__pitcher-name'})
probable_pitchers = []
for i in pitchers:
    probable_pitchers.append(str(i))
#Finding all of the pitchers' names within that <a> tag
#Using for loop to convert the beautiful soup elements in the list to strings so I can get only the pitcher's name from there

In [68]:
probable_pitchers = [i.split('\n')[1] for i in probable_pitchers]
#Getting only the pitchers' names

In [69]:
for i in range(len(probable_pitchers)):
    if 'TBD' in probable_pitchers[i]:
        probable_pitchers[i] = '>TBD</a>'

In [70]:
probable_pitchers

['<a class="probable-pitchers__pitcher-name-link" href="/player/shane-baz-669358">Shane Baz</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/chi-chi-gonzalez-592346">Chi Chi Gonzalez</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/martin-perez-527048">Martin Perez</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/lucas-giolito-608337">Lucas Giolito</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/hunter-greene-668881">Hunter Greene</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/adam-wainwright-425794">Adam Wainwright</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/madison-bumgarner-518516">Madison Bumgarner</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/zack-wheeler-554430">Zack Wheeler</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/eric-lauer-641778">Eric Lauer</a>',
 '<a class="probable-pitchers__pitcher-name-li

In [71]:
probable_pitchers = [i.split('</a')[0] for i in probable_pitchers] 
#Getting only the pitchers' names part 2

In [72]:
probable_pitchers = [i.split('>')[1] for i in probable_pitchers] 

In [74]:
probable_pitchers = [i.split() for i in probable_pitchers]
#Splitting name into first and last name columns

In [77]:
df_probable_pitchers = pd.DataFrame(probable_pitchers)
#Setting list as Pandas DF

In [78]:
df_probable_pitchers = df_probable_pitchers.rename(columns={0: "first_name", 1: "last_name"})
#Renaming column

# Get Pitchers' Teams

In [79]:
new_teams = soup.find_all('span', {'class':['probable-pitchers__team-name probable-pitchers__team-name--away', 'probable-pitchers__team-name probable-pitchers__team-name--home']})
teams_of_probable_pitchers = []
for i in new_teams:
    teams_of_probable_pitchers.append(str(i))
#Finding all of the pitchers' names within that <a> tag
#Using for loop to convert the beautiful soup elements in the list to strings so I can get only the pitcher's name from there

In [80]:
teams_of_probable_pitchers = [x.strip(' ') for x in teams_of_probable_pitchers]
#Getting only the pitchers' names

In [81]:
teams_of_probable_pitchers = [i.split('>\n')[1] for i in teams_of_probable_pitchers] 
#Getting only the pitchers' names

In [82]:
teams_of_probable_pitchers = teams_of_probable_pitchers = [x.strip('                             ') for x in teams_of_probable_pitchers]
#Stripping white space

In [83]:
teams_of_probable_pitchers = [i.split('\n')[0] for i in teams_of_probable_pitchers] 

In [84]:
len(teams_of_probable_pitchers)

32

In [85]:
df_probable_pitchers['team'] = teams_of_probable_pitchers
#Putting team on this way, now we just need to change it to the abbreviation

In [86]:
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Astros', 'HOU', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Nationals', 'WAS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Giants', 'SF', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Blue Jays', 'TOR', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Angels', 'LAA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'White Sox', 'CWS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Guardians', 'CLE', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Athletics', 'OAK', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Twins', 'MIN', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rays', 'TB', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Mariners', 'SEA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Marlins', 'MIA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Padres', 'SD', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Pirates', 'PIT', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Red Sox', 'BOS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Orioles', 'BAL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Braves', 'ATL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rangers', 'TEX', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Phillies', 'PHI', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Mets', 'NYM', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Yankees', 'NYY', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Royals', 'KC', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Cubs', 'CHC', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Brewers', 'MIL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Reds', 'CIN', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rockies', 'COL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Tigers', 'DET', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Dodgers', 'LAD', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'D-backs', 'ARI', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Cardinals', 'STL', df_probable_pitchers['team'])
#Changing team nickname to abbreviation so it will match the other DF -- Will clean this up in the future

In [87]:
misc_pitchers = misc_pitchers[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [88]:
misc_pitchers[['first_name', 'last_name']] = misc_pitchers['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [89]:
misc_pitchers = misc_pitchers.drop(columns='Player')
#We don't need this full name column anymore

In [90]:
misc_pitchers = misc_pitchers.rename(columns={"Team": "team"})
#Make this lowercase so it matches team column in probable pitchers

In [91]:
misc_pitchers = misc_pitchers[['last_name', 'first_name', 'team']]
#reordering columns

# Get Batters' Teams

In [93]:
misc_batters = misc_batters[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [94]:
misc_batters[['first_name', 'last_name']] = misc_batters['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [95]:
misc_batters = misc_batters.drop(columns='Player')
#We don't need this full name column anymore

In [96]:
misc_batters = misc_batters[['last_name', 'first_name', 'Team']]
#Reordering columns

In [97]:
pitchers_df['first_name'] = pitchers_df['first_name'].str.strip()
batters_df['first_name'] = batters_df['first_name'].str.strip()
#Removing empty space in these column entries for example this makes ' Cabrera' => 'Cabrera'

batters_df and pitchers_df front names all have a space in front of them and that's why merge is not working

# Attaching according Team Name to each player in all DFs

The datasets on Baseball Savant do not include the team name with they players, so I will add them in

In [98]:
misc_pitchers = misc_pitchers.drop(misc_pitchers[misc_pitchers.last_name == 'Garcia'].index & misc_pitchers[misc_pitchers.first_name == 'Luis'].index & misc_pitchers[misc_pitchers.team == 'SD'].index)
#Dropping Padres' Luis Garcia to avoid issue of doubling when Astros' Luis Garcia pitches

C:\Users\User\AppData\Local\Temp/ipykernel_3532/877907739.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  misc_pitchers = misc_pitchers.drop(misc_pitchers[misc_pitchers.last_name == 'Garcia'].index & misc_pitchers[misc_pitchers.first_name == 'Luis'].index & misc_pitchers[misc_pitchers.team == 'SD'].index)


In [99]:
pitchers_df = pitchers_df.drop(pitchers_df[pitchers_df.last_name == 'Garcia'].index & pitchers_df[pitchers_df.first_name == 'Luis'].index & pitchers_df[pitchers_df.player_id == 472610 ].index)

C:\Users\User\AppData\Local\Temp/ipykernel_3532/1932908247.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  pitchers_df = pitchers_df.drop(pitchers_df[pitchers_df.last_name == 'Garcia'].index & pitchers_df[pitchers_df.first_name == 'Luis'].index & pitchers_df[pitchers_df.player_id == 472610 ].index)


In [100]:
df_probable_pitchers = pd.merge(df_probable_pitchers, misc_pitchers, on=['last_name', 'first_name', 'team'], how='left')
pitchers_df = pd.merge(pitchers_df, misc_pitchers, on=['last_name', 'first_name'], how='inner') #issue occurs on this merge
batters_df = pd.merge(batters_df, misc_batters, on=['last_name', 'first_name'], how='inner')
#Adding team to all of the players
#Joining probables as a left join so it will include all of the probables without exception -- may make one of their stats
# unavailable if they don't have enough PAs 

In [101]:
df_probable_pitchers = pd.merge(df_probable_pitchers, pitchers_df, on=['team', 'last_name', 'first_name'], how='left')
#Adding stats to probable pitchers

#This is what is adding the other Luis Garcia

# Which player to pick for BTS

In [102]:
pp_xba = df_probable_pitchers[['last_name', 'first_name', 'team', 'batting_avg', 'xba', 'xwoba', 'xslg', 'p_bb_percent']]
#How can I compare this to batters df??

One thing I could do is... switch the one column with the next, and then merge it with the batter's df
So for example if Oakland is playing Cleveland, I can switch the pitchers with each other. So Oakland's pitcher will be on Cleveland, and then I will have the pitcher xba to match with the team. Then I can basically set that for every member of the team, and then add that with the hitter xba and see if it meets the condition

In [103]:
pp_xba.sort_values(by='xba', ascending=False)

,last_name,first_name,team,batting_avg,xba,xwoba,xslg,p_bb_percent
13,Plesac,Zach,CLE,0.273,0.310,0.390,0.565,6.0
9,Corbin,Patrick,WAS,0.313,0.309,0.389,0.525,9.2
17,Brieske,Beau,DET,0.241,0.299,0.403,0.612,8.4
28,Freeland,Kyle,COL,0.281,0.284,0.359,0.492,7.4
19,Morton,Charlie,ATL,0.274,0.280,0.364,0.471,9.9
11,Lynch,Daniel,KC,0.273,0.277,0.347,0.454,10.1
18,Thompson,Zach,PIT,0.265,0.277,0.347,0.468,8.4
25,Montgomery,Jordan,NYY,0.227,0.274,0.324,0.427,4.2
15,Martinez,Nick,SD,0.253,0.267,0.358,0.482,11.3
26,Carrasco,Carlos,NYM,0.255,0.266,0.311,0.409,5.5


In [118]:
batters_df.loc[batters_df['Team'] == 'MIL']

,last_name,first_name,player_id,year,b_ab,b_total_pa,b_total_hits,b_strikeout,b_walk,b_k_percent,...,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,Unnamed: 32,Team,pitcher_xba,xba_total
12,Cain,Lorenzo,456715,2022,130,141,22,34,8,24.1,...,-0.035,88.4,1.6,0.0,80.6,6.3,NaN,MIL,0.309,0.519
13,McCutchen,Andrew,457705,2022,166,180,34,42,11,23.3,...,-0.069,87.8,12.6,7.1,78.7,8.4,NaN,MIL,0.309,0.569
72,Wong,Kolten,543939,2022,174,203,39,33,21,16.3,...,0.013,86.3,16.1,2.1,74.7,9.1,NaN,MIL,0.309,0.536
86,Narvaez,Omar,553882,2022,106,121,29,27,12,22.3,...,0.038,85.2,18.8,5.1,76.5,7.0,NaN,MIL,0.309,0.546
127,Renfroe,Hunter,592669,2022,143,155,38,39,9,25.2,...,0.016,90.0,16.0,14.0,76.3,6.2,NaN,MIL,0.309,0.545
129,Yelich,Christian,592885,2022,193,222,43,57,25,25.7,...,-0.052,92.1,7.1,12.3,75.8,7.0,NaN,MIL,0.309,0.579
166,Caratini,Victor,605170,2022,65,78,14,17,9,21.8,...,-0.051,88.2,5.0,10.2,62.5,7.0,NaN,MIL,0.309,0.592
178,Peterson,Jace,607054,2022,118,133,28,34,14,25.6,...,0.004,88.7,8.1,8.2,58.5,7.0,NaN,MIL,0.309,0.554
203,Taylor,Tyrone,621438,2022,143,154,34,30,8,19.5,...,-0.026,89.3,17.4,8.8,75.0,5.7,NaN,MIL,0.309,0.579
266,Tellez,Rowdy,642133,2022,180,197,46,44,15,22.3,...,-0.048,89.7,15.3,14.0,70.0,6.4,NaN,MIL,0.309,0.588


In [105]:
pp_xba['Team'] = " "
#New column to house when I switch the teams with each other
#I'm doing this so I can eventually merge it onto the batters' df with the hitters' team

C:\Users\User\AppData\Local\Temp/ipykernel_3532/1964202557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_xba['Team'] = " "


# Getting Pitcher Stats onto Batter DF

FOR NOW, UNTIL I CLEAN THE BELOW UP A LITTLE MORE AND HAVE IT ITERATE THROUGH AND ADJUST TO THE NUMBER OF ROWS PER DAY, JUST USE THE AMOUNT OF ROWS THAT THE LEN() SHOWS AND COMMENT OUT THE REST

In [106]:
pp_xba.index = np.arange(1, len(pp_xba) + 1)
#Making index start at 1

In [107]:
len(pp_xba)

32

In [108]:
pp_xba['Team'].iloc[1::2] = pp_xba['team'].iloc[::2]
pp_xba['Team'].iloc[::2] = pp_xba['team'].iloc[1::2]
#Putting odd teams on even teams and vice versa so that 'Team' reflects opposing pitcher

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [109]:
pp_xba = pp_xba.rename(columns={"batting_avg": "pitcher_batting_avg", "xba": "pitcher_xba"})
#Renaming columns for clarity

In [110]:
batters_df = pd.merge(batters_df, pp_xba[['Team','pitcher_xba']],on='Team', how='left')
#Getting pitcher data onto batters df

In [111]:
xba_sum = batters_df['xba'] + batters_df['pitcher_xba']
#Adding sum 

In [112]:
batters_df['xba_total'] = xba_sum
#Adding xba sum into the batters dataframe

# Which Hitters to pick for BTS

# Best Picks for Today

In [113]:
best_picks = batters_df[(batters_df['xba'] > .300) & (batters_df['pitcher_xba'] > .300) & (batters_df['xba_total'] > .650)]
best_picks

,last_name,first_name,player_id,year,b_ab,b_total_pa,b_total_hits,b_strikeout,b_walk,b_k_percent,...,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,Unnamed: 32,Team,pitcher_xba,xba_total


In [114]:
second_best = batters_df[(batters_df['xba'] > .290) & (batters_df['pitcher_xba'] > .290) & (batters_df['xba_total'] > .600)]
second_best

,last_name,first_name,player_id,year,b_ab,b_total_pa,b_total_hits,b_strikeout,b_walk,b_k_percent,...,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,Unnamed: 32,Team,pitcher_xba,xba_total
405,Kirk,Alejandro,672386,2022,137,158,42,14,19,8.9,...,-0.015,90.1,9.7,8.0,75.0,4.4,NaN,TOR,0.299,0.607


In [115]:
third_best = batters_df[(batters_df['xba'] > .275) & (batters_df['pitcher_xba'] > .275) & (batters_df['xba_total'] > .580)]
third_best.sort_values('xba_total', ascending = False)

,last_name,first_name,player_id,year,b_ab,b_total_pa,b_total_hits,b_strikeout,b_walk,b_k_percent,...,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,Unnamed: 32,Team,pitcher_xba,xba_total
405,Kirk,Alejandro,672386,2022,137,158,42,14,19,8.9,...,-0.015,90.1,9.7,8.0,75.0,4.4,NaN,TOR,0.299,0.607
171,Arcia,Orlando,606115,2022,47,54,14,14,6,25.9,...,-0.054,93.8,11.0,11.8,85.0,8.9,NaN,ATL,0.277,0.604
259,Mancini,Trey,641820,2022,194,219,60,38,20,17.4,...,-0.044,89.1,12.7,10.8,75.0,7.2,NaN,BAL,0.277,0.600
46,Bethancourt,Christian,542194,2022,97,105,22,22,6,21.0,...,-0.093,90.5,7.2,10.5,80.0,5.7,NaN,OAK,0.310,0.600
331,Mountcastle,Ryan,663624,2022,169,181,43,44,10,24.3,...,-0.079,91.5,15.6,12.7,89.1,6.4,NaN,BAL,0.277,0.596
392,Espinal,Santiago,669289,2022,175,196,48,34,17,17.3,...,-0.017,87.7,13.7,5.6,73.2,7.4,NaN,TOR,0.299,0.595
375,Bichette,Bo,666182,2022,218,231,56,56,13,24.2,...,-0.038,91.7,8.5,9.9,87.1,8.1,NaN,TOR,0.299,0.593
166,Caratini,Victor,605170,2022,65,78,14,17,9,21.8,...,-0.051,88.2,5.0,10.2,62.5,7.0,NaN,MIL,0.309,0.592
280,Jansen,Danny,643376,2022,50,54,13,9,3,16.7,...,-0.029,89.8,23.4,24.4,65.0,9.6,NaN,TOR,0.299,0.592
364,Guerrero,Vladimir,665489,2022,187,215,47,38,23,17.7,...,-0.032,92.7,5.4,10.6,90.2,6.4,NaN,TOR,0.299,0.589


In [116]:
next_best = batters_df[(batters_df['xba_total'] > .550)]
next_best.sort_values('xba_total', ascending = False)

,last_name,first_name,player_id,year,b_ab,b_total_pa,b_total_hits,b_strikeout,b_walk,b_k_percent,...,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,Unnamed: 32,Team,pitcher_xba,xba_total
216,Reyes,Victor,622682,2022,24,26,7,4,2,15.4,...,-0.093,90.6,4.1,5.0,37.5,7.0,NaN,DET,0.252,0.661
158,Daza,Yonathan,602074,2022,112,126,38,13,9,10.3,...,-0.028,85.5,4.8,4.0,77.4,6.3,NaN,COL,0.267,0.609
405,Kirk,Alejandro,672386,2022,137,158,42,14,19,8.9,...,-0.015,90.1,9.7,8.0,75.0,4.4,NaN,TOR,0.299,0.607
171,Arcia,Orlando,606115,2022,47,54,14,14,6,25.9,...,-0.054,93.8,11.0,11.8,85.0,8.9,NaN,ATL,0.277,0.604
316,Olivares,Edward,658668,2022,35,38,13,5,2,13.2,...,-0.014,92.7,0.6,6.7,90.0,6.6,NaN,KC,0.248,0.603
259,Mancini,Trey,641820,2022,194,219,60,38,20,17.4,...,-0.044,89.1,12.7,10.8,75.0,7.2,NaN,BAL,0.277,0.600
46,Bethancourt,Christian,542194,2022,97,105,22,22,6,21.0,...,-0.093,90.5,7.2,10.5,80.0,5.7,NaN,OAK,0.310,0.600
331,Mountcastle,Ryan,663624,2022,169,181,43,44,10,24.3,...,-0.079,91.5,15.6,12.7,89.1,6.4,NaN,BAL,0.277,0.596
327,Hoerner,Nico,663538,2022,131,137,38,17,4,12.4,...,-0.029,89.7,9.1,4.4,70.0,6.3,NaN,CHC,0.274,0.595
392,Espinal,Santiago,669289,2022,175,196,48,34,17,17.3,...,-0.017,87.7,13.7,5.6,73.2,7.4,NaN,TOR,0.299,0.595
